In [3]:
from langchain.memory import ConversationKGMemory
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0.1)
memory = ConversationKGMemory(llm=llm, return_messages=True)


def add_message(input, output):
    memory.save_context({'input': input}, {'output': output})


add_message("Hi, I'm Flynn. I live in South Korea.", "Wow, that is so cool! I've always wanted to visit South Korea.")

In [5]:
memory.load_memory_variables({'input': 'Who is Flynn?'})

{'history': [SystemMessage(content='On Flynn: Flynn lives in South Korea.')]}

In [6]:
add_message("Flynn loves to travel.", "That's awesome! I love to travel too. Where is your favorite place you've been?")

In [7]:
memory.load_memory_variables({'input': 'What does Flynn love to do?'})

{'history': [SystemMessage(content='On Flynn: Flynn lives in South Korea. Flynn loves to travel.')]}